## Librerias.

In [71]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

## Carga de datos

In [72]:
data_origin = pd.read_excel('Puntualidad Bogotá.xlsx')
datos = data_origin.copy()
datos.head(10)

#datos = datos[:100]

,Fecha / Hora,Asunto
0,2022-06-01T11:07:56+00:00,<7569|20220601> Shein - Control Horas
1,2022-06-01T11:25:34+00:00,<7865|20220601> Messer Consolidado Correos
2,2022-06-01T11:26:32+00:00,<6886|20220601> Messer Ocupación Agente
3,2022-06-01T11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio
4,2022-06-01T11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile
5,2022-06-01T11:37:53+00:00,<7776|20220601> BBVA - BBVA EMPRESAS Recuperac...
6,2022-06-01T11:51:01+00:00,<6885|20220601> Messer 5/15
7,2022-06-01T12:00:26+00:00,<7745|20220601> Pichincha IVR
8,2022-06-01T12:00:30+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...
9,2022-06-01T12:17:21+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...


## Tipos de datos.

In [73]:
datos.dtypes

Fecha / Hora    object
Asunto          object
dtype: object

## Dimenciones del dataframe.

In [74]:
datos.shape

(5383, 2)

## Datos faltantes.

In [75]:
datos_faltantes = datos.isnull().sum().sort_values(ascending=True)
datos_faltantes

Fecha / Hora    0
Asunto          0
dtype: int64

## Porcentajes de datos faltantes por columna.

In [76]:
for column in datos.columns:
    porcentaje_faltantes = datos[column].isnull().mean()
    print(column, '=', round(porcentaje_faltantes*100,2))

Fecha / Hora = 0.0
Asunto = 0.0


## Convertir columna tipo object a datetime64 en una nueva columna.

In [77]:
datos['Datetime'] = pd.to_datetime(datos['Fecha / Hora'])

datos.dtypes

Fecha / Hora                 object
Asunto                       object
Datetime        datetime64[ns, UTC]
dtype: object

In [78]:
datos.head(10)

,Fecha / Hora,Asunto,Datetime
0,2022-06-01T11:07:56+00:00,<7569|20220601> Shein - Control Horas,2022-06-01 11:07:56+00:00
1,2022-06-01T11:25:34+00:00,<7865|20220601> Messer Consolidado Correos,2022-06-01 11:25:34+00:00
2,2022-06-01T11:26:32+00:00,<6886|20220601> Messer Ocupación Agente,2022-06-01 11:26:32+00:00
3,2022-06-01T11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio,2022-06-01 11:30:51+00:00
4,2022-06-01T11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile,2022-06-01 11:33:21+00:00
5,2022-06-01T11:37:53+00:00,<7776|20220601> BBVA - BBVA EMPRESAS Recuperac...,2022-06-01 11:37:53+00:00
6,2022-06-01T11:51:01+00:00,<6885|20220601> Messer 5/15,2022-06-01 11:51:01+00:00
7,2022-06-01T12:00:26+00:00,<7745|20220601> Pichincha IVR,2022-06-01 12:00:26+00:00
8,2022-06-01T12:00:30+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...,2022-06-01 12:00:30+00:00
9,2022-06-01T12:17:21+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...,2022-06-01 12:17:21+00:00


## Identificar los ID.

In [79]:
datos['ID'] = 0
patron = r'<(\d{4})|'
for i in range(len(datos)):
    datos['ID'][i] = re.findall(patron,str(datos.Asunto[i].strip()))[0]

/tmp/ipykernel_3170/1160141558.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['ID'][i] = re.findall(patron,str(datos.Asunto[i].strip()))[0]
/usr/lib/python3/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [80]:
datos

,Fecha / Hora,Asunto,Datetime,ID
0,2022-06-01T11:07:56+00:00,<7569|20220601> Shein - Control Horas,2022-06-01 11:07:56+00:00,7569
1,2022-06-01T11:25:34+00:00,<7865|20220601> Messer Consolidado Correos,2022-06-01 11:25:34+00:00,7865
2,2022-06-01T11:26:32+00:00,<6886|20220601> Messer Ocupación Agente,2022-06-01 11:26:32+00:00,6886
3,2022-06-01T11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio,2022-06-01 11:30:51+00:00,6887
4,2022-06-01T11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile,2022-06-01 11:33:21+00:00,6887
...,...,...,...,...
5378,2022-06-30T20:00:10+00:00,<7408|20220630> BBVA - Reporte Tipificación Ou...,2022-06-30 20:00:10+00:00,7408
5379,2022-06-30T20:00:46+00:00,<8061|20220630> Cruz Verde Tablero De Control ...,2022-06-30 20:00:46+00:00,8061
5380,2022-06-30T20:01:10+00:00,<6874|20220630> Cruz Verde Actualización Kpi's...,2022-06-30 20:01:10+00:00,6874
5381,2022-06-30T20:01:21+00:00,<7689|20220630> Medicarte Tablero De Control J...,2022-06-30 20:01:21+00:00,7689
